__Imports__

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from Scripts.Individual import Individual
from Scripts.Parameters import mu, m, N, pa
from Scripts.Polarity import polarity
from Scripts.Memory import initialize_memory
from Scripts.Model import Model
from multiprocessing import Pool
from time import time

seed = 42

In [3]:
kappa = 10
alpha = 0.5
omega = 0.5
lambd = 5
gamma = 5.0

In [4]:
model = Model(N, pa, mu, m, kappa, lambd, alpha, omega, gamma, seed)

In [9]:
def f(node):
    model.indInfo(node).update_memory()
    
pool = Pool(processes = 2)

In [ ]:
start = time()
pool.map(f, range(model.N))
end   = time()
print(end - start)

In [7]:
start = time()
model.update_memory()
end   = time()
print(end - start)

1.0083873271942139


In [8]:
memory = initialize_memory(mu, m)

In [12]:
polarity(memory[0])

0.6190424201551366

In [34]:
pool = Pool(8)

In [35]:
start = time()
L1 = list(map(polarity, memory))
end = time()
print(end - start)

0.0029916763305664062


In [37]:
start = time()
L2 = pool.map(polarity, memory)
end = time()
print(end - start)

0.008949518203735352


In [45]:
individual = Individual(mu, m, kappa)

In [46]:
start = time()
L3 = individual.compute_polarization()
end = time()
print(end - start)

0.8367629051208496


***

## Algoritmo

_O grafo será inicializado com a memória de todos os indivíduos totalmente preenchida aleatoriamente, a fim de que seja possível calcular os valores iniciais para os pesos das arestas. A memória funcionará como uma estrutura de fila do tipo $ FIFO $, first-in-first-out, onde a inserção de uma nova informação na fila desloca as informações mais antigas, descartando a última da fila. Se definirmos $\varphi(\cdot)$ como um operador que converte números inteiros em suas representações binárias, então o estado inicial da memória de um vértice $u$, $L_{u}(0)$, é dado por_

$$
\begin{equation}
  L_{u}(0) = \{x_{i}\; |\; i = 1, \dots, \mu;\;\; x_{i} = \varphi(X),\;\;  X \thicksim  B (2^{m}, 0.5) \}
\end{equation}
$$

_onde $B(2^{m}, 0.5)$ é uma distribuição binomial com parâmetros $(n, p) = (2^{n}, 0.5)$, aproximando uma distribuição normal com parâmetros $(\mu, \sigma^2) = (2^{n - 1}, 2^{n - 2})$. É cabível observar que os parâmetro definidos dependem da memória dos indivíduos em cada instante, que é alterada ao longo da simulação. Há, portanto, uma dependência temporal desses parâmetros. Por simplicidade omitiremos na notação a consideração de que as definições e processos descritos nesta seção dizem respeito a um dado instante $t$._

_Também é feita a especificação dos parâmetros $\alpha$ e $\omega$ a fim de dividir os vértices da rede nos grupos $V_{\downarrow}$, $V_{\uparrow}$ e $\overline{V}$, que determina o padrão de polarização de cada vértice._ 

_Em uma primeira fase da simulação, admitindo que exista uma representação do grafo por uma lista de arestas $E$, o algoritmo percorrerá todos os elementos $\langle u, v \rangle \in E$ realizando trocas de informações entre $u$ e $v$, i.e. escolhe-se ao acaso $x \in X_{u}$ e $y \in X_{v}$ partindo das distribuições $P_{u}$ e $P_{v}$, e faz-se a transmissão de $x$ para $v$ e $y$ para $u$ levando em consideração as probabilidades de distorção definidas anteriormente para os indivíduos de cada grupo, isto é, considerando que o indivíduo possa polarizar para cima, para baixo ou se manter neutro. Portanto, a informação recebida por um indivíduo não corresponde necessariamente à informação emitida. Nesse processo, também é garantido que todo vértice transmitirá alguma informação (não necessariamente a mesma) para cada um dos seus vizinhos._

_Após a seleção se verifica a aceitação da informação emitida, segundo a probabilidade $\eta_{u \to  v}$ definida acima. As informações rejeitadas são descartadas, enquanto que as informações aceitas são armazenadas em uma lista temporária até a conclusão da disseminação na rede, em que os dados da rede são atualizados._

_Ao término de um momento $t$ da simulação, cada vértice $u \in V(G)$ passa a ter uma nova memória $L_{u}(t + 1)$ com a adição das informações recebidas, e o grafo pode ser atualizado com o cálculo dos novos pesos $JSD(u, v)$ e dos parâmetros que dependem deles._

#### Modos de avaliar o modelo.

- Calcular estatísticas globais e plotar sua evolução no tempo.
- Plotar a distribuição de estatísticas locais em certos pontos da simulação.
- Dinâmica avaliando em paralelo a proporção de grupos.

#### Modos de melhorar o modelo.
- Avaliar o custo de processamento das diferentes partes da simulação; identificar pontos passíveis de otimização.


### Para fazer
- Paralelize o cálculo da entropia nos indivíduos.
- Faça uma função para retornar uma figura pré-formatada para plotar curvas. Padronize a formatação desses gráficos.
- Faça funções para plotar as estatísticas de maneira organizada.